In [36]:
import pandas as pd

relevant = pd.read_json('~/Downloads/relevant_news_10K.json');
relevant['index'] = relevant.index
relevant['label']=1
irrelevant = pd.read_json('~/Downloads/irrelevant_news_10K.json')
irrelevant['index']=irrelevant.index
irrelevant['label']=0
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

UsageError: Line magic function `%install_ext` not found.


In [42]:
relevant.head(20)

,content,headline,source,summary,uid,index,label
0,The lowly milkshake has turned into an unlikel...,Milkshakes become weapon of choice in UK Europ...,Agence France Presse,Former UK Independence Party leader Nigel Fara...,a437ff48-104a-54bb-bff7-c7a736158524,0,1
1,ANZ has moved to the front of the race that’s ...,ANZ's first assault in the looming job armageddon,News Ltd.,ANZ has moved to the front of the race that’ s...,366c92af-8143-5ffa-8702-4f26bd22c8b6,1,1
2,Jul. 10--Carnival Cruise Line stateroom attend...,Carnival Cruise Line to collect your used soap...,Tribune Content Agency,Jul. 10-- Carnival Cruise Line stateroom atten...,863096d4-48f0-5a7c-bee6-384a76d575ee,2,1
3,CHENNAI: R Rohit (5/43) and P Saravanan (5/46)...,Standard CC bags fourth title in a row [New In...,SyndiGate Media Inc.,CHENNAI: R Rohit and P Saravanan took five wic...,3e4d6490-4224-595e-be26-4cb249209b8f,3,1
4,Donald Trump’s nominee to lead the US Fish and...,Revealed: Trump's Wildlife Service pick has ti...,Guardian,New revelations show she also has ties to the ...,9f3e248d-b040-5058-bdc9-61c4de59f02a,4,1
5,"SAN FRANCISCO, May 30, 2019 /PRNewswire/ -- Ca...",Castlight Health to Participate in Upcoming In...,PR Newswire,"--Castlight Health, Inc., a leading health ben...",c058dcb3-87b0-5ffe-80f7-92506adcdff4,5,1
6,"Seventy-five years ago, President Franklin Del...",Celebrating the pursuit of scientic innovation,"Boston Globe, The","Seventy-five years ago, President Franklin Del...",f69f3ef1-d5f8-5202-a1ff-733316dcd490,6,1
7,"GARDEN GROVE, Calif., Aug. 26, 2019 /PRNewswir...","HRH Dr. Percy Tamayo Joins C3 International, I...",PR Newswire,Dr. Tamayo is the current Secretary General an...,19d67e93-4aed-5b37-8a1d-c35cb02d975a,7,1
8,"EAST RUTHERFORD, N.J. (AP) — More than two dec...",Massive American Dream mall to open but will s...,"Associated Press, The",—More than two decades ago when a mega enterta...,1436c165-01f6-5b18-ab13-7f9a3790f01f,8,1
9,Oct. 31--Ford Motor Co. has gone from beating ...,"Ford's former violent, thuggish relationship w...",Tribune Content Agency,The modern relationship culminated Wednesday i...,406b2404-50a6-59cc-9443-82fac2af831e,9,1


In [38]:
(relevant[0:10]['content'])

0    The lowly milkshake has turned into an unlikel...
1    ANZ has moved to the front of the race that’s ...
2    Jul. 10--Carnival Cruise Line stateroom attend...
3    CHENNAI: R Rohit (5/43) and P Saravanan (5/46)...
4    Donald Trump’s nominee to lead the US Fish and...
5    SAN FRANCISCO, May 30, 2019 /PRNewswire/ -- Ca...
6    Seventy-five years ago, President Franklin Del...
7    GARDEN GROVE, Calif., Aug. 26, 2019 /PRNewswir...
8    EAST RUTHERFORD, N.J. (AP) — More than two dec...
9    Oct. 31--Ford Motor Co. has gone from beating ...
Name: content, dtype: object

In [59]:
import numpy as np
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
import multiprocessing as mp
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator
from normalise import normalise
import pandas as pd

stopWords = ["nt","'ll", "'ve", '1-1', 'a', "a's", 'able', 'about', 'above', 'abroad', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'adopted', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ago', 'ah', 'ahead', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'apparently', 'appear', 'appreciate', 'appropriate', 'approximately', 'are', 'area', 'areas', 'aren', "aren't", 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asked', 'asking', 'asks', 'associated', 'at', 'auth', 'available', 'away', 'awfully', 'b', 'back', 'backed', 'backing', 'backs', 'backward', 'backwards', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'began', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'beings', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'big', 'bill', 'biol', 'both', 'bottom', 'brief', 'briefly', 'but', 'by', 'c', "c'mon", "c's", 'ca', 'call', 'called', 'came', 'can', "can't", 'cannot', 'cant', 'caption', 'case', 'cases', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clear', 'clearly', 'co', 'co.', 'com', 'come', 'comes', 'computer', 'con', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', "couldn't", 'couldnt', 'course', 'cry', 'currently', 'd', 'dare', "daren't", 'date', 'de', 'dear', 'definitely', 'describe', 'described', 'despite', 'detail', 'did', "didn't", 'differ', 'different', 'differently', 'directly', 'do', 'does', "doesn't", 'doing', "don't", 'done', 'down', 'downed', 'downing', 'downs', 'downwards', 'due', 'during', 'e', 'each', 'early', 'ed', 'edu', 'effect', 'eg', 'eight', 'eighty', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'end', 'ended', 'ending', 'ends', 'enough', 'entirely', 'especially', 'et', 'et-al', 'etc', 'even', 'evenly', 'ever', 'evermore', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'f', 'face', 'faces', 'fact', 'facts', 'fairly', 'far', 'farther', 'felt', 'few', 'fewer', 'ff', 'fifteen', 'fifth', 'fify', 'fill', 'find', 'finds', 'fire', 'first', 'five', 'fix', 'followed', 'following', 'follows', 'for', 'forever', 'former', 'formerly', 'forth', 'forty', 'forward', 'found', 'four', 'from', 'front', 'full', 'fully', 'further', 'furthered', 'furthering', 'furthermore', 'furthers', 'g', 'gave', 'general', 'generally', 'get', 'gets', 'getting', 'give', 'given', 'gives', 'giving', 'go', 'goes', 'going', 'gone', 'good', 'goods', 'got', 'gotten', 'great', 'greater', 'greatest', 'greetings', 'group', 'grouped', 'grouping', 'groups', 'h', 'had', "hadn't", 'half', 'happens', 'hardly', 'has', "hasn't", 'hasnt', 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'hed', 'held', 'hello', 'help', 'hence', 'her', 'here', "here's", 'hereafter', 'hereby', 'herein', 'heres', 'hereupon', 'hers', 'herse', 'herself', 'hes', 'hi', 'hid', 'high', 'higher', 'highest', 'him', 'himse', 'himself', 'his', 'hither', 'home', 'hopefully', 'how', 'howbeit', 'however', 'hundred', 'i', "i'd", "i'll", "i'm", "i've", 'id', 'ie', 'if', 'ignored', 'im', 'immediate', 'immediately', 'importance', 'important', 'in', 'inasmuch', 'inc', 'inc.', 'include', 'included', 'including', 'indeed', 'index', 'indicate', 'indicated', 'indicates', 'information', 'inner', 'inside', 'insofar', 'instead', 'interest', 'interested', 'interesting', 'interests', 'into', 'invention', 'inward', 'is', "isn't", 'it', "it'd", "it'll", "it's", 'itd', 'its', 'itse', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kept', 'keys', 'kg', 'kind', 'km', 'knew', 'know', 'known', 'knows', 'l', 'large', 'largely', 'last', 'late', 'lately', 'later', 'latest', 'latter', 'latterly', 'least', 'led', 'less', 'lest', 'let', "let's", 'lets', 'like', 'liked', 'likely', 'likewise', 'line', 'links', 'little', 'long', 'longer', 'longest', 'look', 'looking', 'looks', 'low', 'lower', 'ltd', 'm', 'made', 'mainly', 'make', 'makes', 'making', 'man', 'many', 'may', 'maybe', "mayn't", 'me', 'mean', 'means', 'meantime', 'meanwhile', 'member', 'members', 'men', 'merely', 'mg', 'might', "mightn't", 'mill', 'million', 'mine', 'minus', 'miss', 'ml', 'more', 'moreover', 'most', 'mostly', 'move', 'moved', 'mr', 'mrs', 'much', 'mug', 'must', "mustn't", 'my', 'myse', 'myself', 'n', 'na', 'name', 'namely', 'nay', 'nd', 'near', 'nearly', 'necessarily', 'necessary', 'need', 'needed', 'needing', "needn't", 'needs', 'neither', 'never', 'neverf', 'neverless', 'nevertheless', 'new', 'newer', 'newest', 'next', 'nine', 'ninety', 'no', 'no-one', 'nobody', 'non', 'none', 'nonetheless', 'noone', 'nor', 'normally', 'nos', 'not', 'noted', 'nothing', 'notwithstanding', 'novel', 'now', 'nowhere', 'number', 'numbers', 'o', 'obtain', 'obtained', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'older', 'oldest', 'omitted', 'on', 'once', 'one', "one's", 'ones', 'only', 'onto', 'open', 'opened', 'opening', 'opens', 'opposite', 'or', 'ord', 'order', 'ordered', 'ordering', 'orders', 'other', 'others', 'otherwise', 'ought', "oughtn't", 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'owing', 'own', 'p', 'page', 'pages', 'part', 'parted', 'particular', 'particularly', 'parting', 'parts', 'past', 'per', 'perhaps', 'place', 'placed', 'places', 'please', 'plus', 'point', 'pointed', 'pointing', 'points', 'poorly', 'possible', 'possibly', 'potentially', 'pp', 'predominantly', 'present', 'presented', 'presenting', 'presents', 'presumably', 'previously', 'primarily', 'probably', 'problem', 'problems', 'promptly', 'proud', 'provided', 'provides', 'put', 'puts', 'q', 'que', 'quickly', 'quite', 'qv', 'r', 'ran', 'rather', 'rd', 're', 'readily', 'really', 'reasonably', 'received', 'recent', 'recently', 'ref', 'refs', 'regarding', 'regardless', 'regards', 'related', 'relatively', 'research', 'respectively', 'resulted', 'resulting', 'results', 'right', 'room', 'rooms', 'round', 'run', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'sec', 'second', 'secondly', 'seconds', 'section', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'sees', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', "shan't", 'she', "she'd", "she'll", "she's", 'shed', 'shes', 'should', "shouldn't", 'show', 'showed', 'showing', 'shown', 'showns', 'shows', 'side', 'sides', 'significant', 'significantly', 'similar', 'similarly', 'since', 'sincere', 'six', 'sixty', 'slightly', 'small', 'smaller', 'smallest', 'so', 'some', 'somebody', 'someday', 'somehow', 'someone', 'somethan', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specifically', 'specified', 'specify', 'specifying', 'state', 'states', 'still', 'stop', 'strongly', 'sub', 'substantially', 'successfully', 'such', 'sufficiently', 'suggest', 'sup', 'sure', 'system', 't', "t's", 'take', 'taken', 'taking', 'tell', 'ten', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', "that'll", "that's", "that've", 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', "there'd", "there'll", "there're", "there's", "there've", 'thereafter', 'thereby', 'thered', 'therefore', 'therein', 'thereof', 'therere', 'theres', 'thereto', 'thereupon', 'these', 'they', "they'd", "they'll", "they're", "they've", 'theyd', 'theyre', 'thick', 'thin', 'thing', 'things', 'think', 'thinks', 'third', 'thirty', 'this', 'thorough', 'thoroughly', 'those', 'thou', 'though', 'thoughh', 'thought', 'thoughts', 'thousand', 'three', 'throug', 'through', 'throughout', 'thru', 'thus', 'til', 'till', 'time', 'tip', 'tis', 'to', 'today', 'together', 'too', 'took', 'top', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'ts', 'turn', 'turned', 'turning', 'turns', 'twas', 'twelve', 'twenty', 'twice', 'two', 'u', 'un', 'under', 'underneath', 'undoing', 'unfortunately', 'unless', 'unlike', 'unlikely', 'until', 'unto', 'up', 'upon', 'ups', 'upwards', 'us', 'use', 'used', 'useful', 'usefully', 'usefulness', 'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'versus', 'very', 'via', 'viz', 'vol', 'vols', 'vs', 'w', 'want', 'wanted', 'wanting', 'wants', 'was', "wasn't", 'way', 'ways', 'we', "we'd", "we'll", "we're", "we've", 'wed', 'welcome', 'well', 'wells', 'went', 'were', "weren't", 'what', "what'll", "what's", "what've", 'whatever', 'whats', 'when', 'whence', 'whenever', 'where', "where's", 'whereafter', 'whereas', 'whereby', 'wherein', 'wheres', 'whereupon', 'wherever', 'whether', 'which', 'whichever', 'while', 'whilst', 'whim', 'whither', 'who', "who'd", "who'll", "who's", 'whod', 'whoever', 'whole', 'whom', 'whomever', 'whos', 'whose', 'why', 'widely', 'will', 'willing', 'wish', 'with', 'within', 'without', "won't", 'wonder', 'words', 'work', 'worked', 'working', 'works', 'world', 'would', "wouldn't", 'written', 'www', 'x', 'y', 'year', 'years', 'yes', 'yet', 'you', "you'd", "you'll", "you're", "you've", 'youd', 'young', 'younger', 'youngest', 'your', 'yourabout', 'youre', 'yours', 'yourself', 'yourselves', 'z', 'zero']


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 variety="BrE",
                 user_abbrevs={},
                 n_jobs=1):
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        len(data)
        pool.close()
        pool.join()

        return data
    
    def strip_html(self,text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    def remove_between_square_brackets(self,text):
        return re.sub('\[[^]]*\]', '', text)

    def denoise_text(self,text):
        text = self.strip_html(text)
        text = self.remove_between_square_brackets(text)
        return text
    
    def remove_non_ascii(self,words):
    
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words

    def to_lowercase(self,words):
        """Convert all characters to lowercase from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = word.lower()
            new_words.append(new_word)
        return new_words

    def remove_punctuation(self,words):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words

    def replace_numbers(self,words):
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        p = inflect.engine()
        new_words = []
        for word in words:
            if word.isdigit():
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            else:
                new_words.append(word)
        return new_words

    def remove_stopwords(self,words):
        """Remove stop words from list of tokenized words"""
        new_words = []
        for word in words:
            if word not in stopwords.words('english') and word not in stopWords:
                new_words.append(word)
        return new_words

    def stem_words(self,words):
        """Stem words in list of tokenized words"""
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        return stems

    def lemmatize_verbs(self,words):
        """Lemmatize verbs in list of tokenized words"""
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return lemmas

    def normalize2(self,words):
        words = self.remove_non_ascii(words)
        words = self.remove_punctuation(words)
        words = self.to_lowercase(words)
#         words = self.replace_numbers(words)
        words = self.remove_stopwords(words)
        return words

    def stem_and_lemmatize(self,words):
        #stems = stem_words(words)
        lemmas = self.lemmatize_verbs(words)
        return lemmas

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        sample = text
        print(sample)
        words = nltk.word_tokenize(sample)
        print(words)
        words = self.normalize2(words)
        
        words = self.stem_and_lemmatize(words)
        print(words)
        return ' '.join(words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop and t.doc not in stopWords and t.doc not in ["http","multimedia","html"]]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])

In [60]:
%%time
#relevant['content'][1:2] =
print(relevant['content'][1:2])
summary = pd.Series(["ANZ hås moved to the front of the Råce that’s set to bring åbout the biggest White-Collår job loss revolution in our history. ---"])
x = TextPreprocessor(n_jobs=10).transform(summary)
#y = TextPreprocessor(n_jobs=10).transform(relevant['content'])

1    ANZ has moved to the front of the race that’s ...
Name: content, dtype: object
ANZ hås moved to the front of the Råce that’s set to bring åbout the biggest White-Collår job loss revolution in our history. ---
['ANZ', 'hås', 'moved', 'to', 'the', 'front', 'of', 'the', 'Råce', 'that', '’', 's', 'set', 'to', 'bring', 'åbout', 'the', 'biggest', 'White-Collår', 'job', 'loss', 'revolution', 'in', 'our', 'history', '.', '--', '-']
['ANZ', 'has', 'moved', 'to', 'the', 'front', 'of', 'the', 'Race', 'that', '', 's', 'set', 'to', 'bring', 'about', 'the', 'biggest', 'White-Collar', 'job', 'loss', 'revolution', 'in', 'our', 'history', '.', '--', '-']
['ANZ', 'has', 'moved', 'to', 'the', 'front', 'of', 'the', 'Race', 'that', 's', 'set', 'to', 'bring', 'about', 'the', 'biggest', 'WhiteCollar', 'job', 'loss', 'revolution', 'in', 'our', 'history']
['anz', 'has', 'moved', 'to', 'the', 'front', 'of', 'the', 'race', 'that', 's', 'set', 'to', 'bring', 'about', 'the', 'biggest', 'whitecollar', 'job', '

In [1]:
x

NameError: name 'x' is not defined

In [14]:
irrelevant['content'] = x
irrelevant.to_json("~/irrelevantNewsNLTK4.json")

relevant['content'] = y
relevant.to_json("~/relevantNewsNLTK4.json")